## The notebook was executed in Google Collab

In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!cp '/content/drive/MyDrive/Colab Notebooks/kaggle.json' ~/.kaggle/
!kaggle datasets download bollywood-celeb-localized-face-dataset-extended
!unzip '/content/bollywood-celeb-localized-face-dataset-extended.zip' -d '/content'

In [ ]:
!pip install mtcnn==0.1.0 tensorflow==2.3.1 keras==2.4.3 keras-vggface==0.6 keras_applications==1.0.8 streamlit
!npm install localtunnel

In [ ]:
import os
import numpy as np
import pandas as pd

# from tensorflow.keras.preprocessing import image
from keras.preprocessing import image
from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace

from tqdm import tqdm
import pickle
from mtcnn import MTCNN
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
import cv2
from google.colab.patches import cv2_imshow

import warnings
warnings.filterwarnings('ignore')

Change code in the `usr/local/lib/python3.7/dist-packages/keras_vggface/models.py` file from

`from keras.engine.topology import get_source_inputs`

TO

`from keras.utils.layer_utils import get_source_inputs`

[Error fix for VGGFace Import](https://shubhasmitaroy.medium.com/for-those-who-get-an-error-on-importing-vggface-related-packages-e5afdd0c3f01)

In [ ]:
actors = sorted(os.listdir('dataset'))
filenames = []
for actor in actors:
  for file in os.listdir(os.path.join('dataset', actor)):
    filenames.append(os.path.join('dataset', actor, file))

pickle.dump(filenames, open('filenames.pkl', 'wb'))
print(f"Total Files: {len(filenames)}\n")
filenames[:5]

Total Files: 12257



['dataset/Aamir_Khan/Aamir.234.jpg',
 'dataset/Aamir_Khan/Aamir.112.jpg',
 'dataset/Aamir_Khan/Aamir.85.jpg',
 'dataset/Aamir_Khan/Aamir.74.jpg',
 'dataset/Aamir_Khan/Aamir.559.jpg']

In [ ]:
def feature_extractor(img_path, model):
  img = image.load_img(img_path, target_size=(224,224))
  img_array = image.img_to_array(img)
  expanded_img = np.expand_dims(img_array, axis=0)
  preprocessed_img = preprocess_input(expanded_img)
  result = model.predict(preprocessed_img).flatten()
  return result
  
model = VGGFace(model='resnet50',
                include_top=False,
                input_shape=(224,224,3),
                pooling='avg'
                )
model.summary()

In [ ]:
# extracting feature
features = []
for file in tqdm(filenames):
  features.append(feature_extractor(file, model))

# save the pickle file
pickle.dump(features, open('embedding.pkl', 'wb'))

100%|██████████| 12257/12257 [40:23<00:00,  5.06it/s]


In [ ]:
import sys

# load image & extract its face & its features
# find the cosine distance of current image with all the features
# recommend that image
feature_list = np.array(pickle.load(open('/content/drive/MyDrive/Colab Notebooks/Dataset/celeb_face_embedding.pkl', 'rb')))
model = VGGFace(model='resnet50',
                include_top=False,
                input_shape=(224,224,3),
                pooling='avg'
                )

IMAGE_PATH = "/content/drive/MyDrive/Colab Notebooks/Dataset/sample_img/"
sample_img = cv2.imread(IMAGE_PATH+"r3.jpg")

detector = MTCNN()
results = detector.detect_faces(sample_img)
if len(results) == 0:
  print('[ERROR] No face detected',results)
  sys.exit()

x,y,w,h = results[0]['box']

# feature extraction
face = sample_img[y:y+h, x:x+w]
image = Image.fromarray(face)
image = image.resize((224,224))
face_array = np.asarray(image)
face_array = face_array.astype('float32')
expanded_img = np.expand_dims(face_array, axis=0)
preprocessed_img = preprocess_input(expanded_img)
result = model.predict(preprocessed_img).flatten()

# comparison using cosine similarity
similarity = []
for i in range(len(feature_list)):
  similarity.append(cosine_similarity(result.reshape(1,-1), feature_list[i].reshape(1,-1))[0][0])

index_pos = sorted(list(enumerate(similarity)), reverse=True, key=lambda x: x[1])[0][0]
temp_img = cv2.imread(filenames[index_pos])

# cv2.imshow('output', face)
cv2_imshow(temp_img)